In [1]:
# divide-word.py
from collections import defaultdict
import math

# データパス
model_path = "../../test/04-model.txt"
input_path = "../../test/04-input.txt"

uni_probs = defaultdict(lambda: 0)
# unigramのモデルを読み込み
with open(model_path) as f:
    for line in f:
        unigram, prob = line.split()
        uni_probs[unigram] = float(prob)

with open(input_path) as f:
    # 前向きステップ
    for line in f:
        
        best_edge = [None] * len(line)
        best_score = [0] * len(line)
        for word_end in range(1, len(line)):
            best_score[word_end] = 1e10
            for word_begin in range(word_end):
                print(word_begin, word_end)
                word = line[word_begin : word_end]
                print(word)
                if word in uni_probs.keys() or len(word) == 1:
                    prob = uni_probs[word]
                    my_score = best_score[word_begin] - math.log(prob, 2)
                    if my_score < best_score[word_end]:
                        best_score[word_end] = my_score
                        best_edge[word_end] = (word_begin, word_end)
        '''
        ex.
        best_score = [0, 5.770780162668462, 11.541560325336924, 8.80043975151574]
        best_edge = [None, (0, 1), (1, 2), (1, 3)]
        '''
        words = []
        next_edge = best_edge[len(best_edge) - 1]
        while next_edge:
            # このエッジの部分文字列を追加
            word = line[next_edge[0]:next_edge[1]]
            words.append(word)
            next_edge = best_edge[next_edge[0]]
        words = words[::-1]
        print("\t".join(words))

0 1
a
0 2
ab
1 2
b
0 3
abc
1 3
bc
2 3
c
ab	c
0 1
b
0 2
bb
1 2
b
0 3
bbc
1 3
bc
2 3
c
b	bc


In [15]:
# divide-word.py
from collections import defaultdict
import math

UNKNOWN_RATE = 0.05
N = 1000000

word_data_path = "../../data/wiki-ja-train.word"
save_model_path = "wiki-ja-train-model.txt"
input_path = "../../data/wiki-ja-test.txt"
output_path = "tutorial03.txt"
# ディクショナリの初期値を0に設定
d = defaultdict(lambda: 0)
total_count = 0

# テキストの読み込み
with open(word_data_path) as f:
    for line in f:
        words = line.split()
        words.append("</s>")
        # 特定の単語の出現数と全体の単語数をカウント
        for word in words:
            d[word] += 1
            total_count += 1

# 単語と出現率を出力
with open(save_model_path, mode='w') as f:
    for key, value in d.items():
        prob = value / total_count
        f.write("{} {}\n".format(key, prob))


uni_probs = defaultdict(lambda: 0)
# unigramのモデルを読み込み
with open(save_model_path) as f:
    for line in f:
        unigram, prob = line.split()
        uni_probs[unigram] = float(prob)

with open(input_path) as f, open(output_path, mode="w") as fw:
    # 前向きステップ
    for sentence in f:
        # edgeとscoreの初期化
        best_edge = [None] * len(sentence)
        best_score = [0] * len(sentence)
        # 1, 2, 3,..., N文字目までの最適な経路を求めていく
        for word_end in range(1, len(sentence)):
            # n文字目までのスコアを大きく取っておき、小さい時に更新する
            best_score[word_end] = 1e10
            # 開始エッジ(word_begin)
            for word_begin in range(word_end):
                # 文の開始位置と終端位置から候補の文字を出す
                word = sentence[word_begin : word_end]
                # wordは、 A, AB, B, ABC, BC, Cのように更新されていく
                # wordがユニグラムモデルに入っていない場合か、単語が一文字の時は飛ばす
                if not(word in uni_probs.keys() or len(word) == 1):
                    continue
                # その単語が出る確率を計算(unknown対策もする)
                prob = (1 - UNKNOWN_RATE) * uni_probs[word] + (UNKNOWN_RATE / N)
                # 開始単語に着くまでのスコアと今回の単語のスコアを足す
                # -logなので、確率が低い程スコアが大きくなり、確率が高い程スコアは変化しない
                my_score = best_score[word_begin] - math.log(prob, 2)
                # スコアが改善された場合、代入する
                if my_score < best_score[word_end]:
                    best_score[word_end] = my_score
                    best_edge[word_end] = (word_begin, word_end)
        '''
        ex.
        best_score = [0, 5.770780162668462, 11.541560325336924, 8.80043975151574]
        best_edge = [None, (0, 1), (1, 2), (1, 3)]
        '''
        # 前からそのエッジに着くまでの最適経路を出しているので、逆から辿れば終端までの最適経路がわかる
        # 初期化
        words = []
        next_edge = best_edge[len(best_edge) - 1]
        # 最後のエッジ(0)にはNoneが入っているので、Noneが来るまで
        while next_edge:
            # このエッジの部分文字列を追加
            word = sentence[next_edge[0]:next_edge[1]]
            words.append(word)
            next_edge = best_edge[next_edge[0]]
        # 終端から辿っているので逆順に直す
        words = words[::-1]
        # 単語分割をスペース区切りで出力
        out_put_str = " ".join(words)
        fw.write(out_put_str + "\n")

'''
$ ../../script/gradews.pl ../../data/wiki-ja-test.word tutorial03.txt
Sent Accuracy: 23.81% (20/84)
Word Prec: 71.88% (1943/2703)
Word Rec: 84.22% (1943/2307)
F-meas: 77.56%
Bound Accuracy: 86.30% (2784/3226)
'''

'\n$ ../../script/gradews.pl ../../data/wiki-ja-test.word tutorial03.txt\nSent Accuracy: 23.81% (20/84)\nWord Prec: 71.88% (1943/2703)\nWord Rec: 84.22% (1943/2307)\nF-meas: 77.56%\nBound Accuracy: 86.30% (2784/3226)\n'

In [16]:
!../../script/gradews.pl ../../data/wiki-ja-test.word tutorial03.txt

Sent Accuracy: 23.81% (20/84)
Word Prec: 71.88% (1943/2703)
Word Rec: 84.22% (1943/2307)
F-meas: 77.56%
Bound Accuracy: 86.30% (2784/3226)


In [46]:
!cat  ../../test/04-model.txt

a	0.0907179533
b	0.0183156389
c	0.100258844
ab	0.246596964
bc	0.122456428


In [3]:
!cat ../../test/04-answer.txt

ab c
b bc


In [6]:
!cat ../../data/wiki-ja-test.txt | head -5

校正（こうせい、英：ｐｒｏｏｆｒｅａｄｉｎｇ、中：校対）は、印刷物等の字句や内容、体裁、色彩の誤りや不具合を、あらかじめ修正すること。
校合（きょうごう）ともいう。
出版にあたっては、印刷に先立って仮刷りを行い、それと原稿の内容を突き合わせ、誤植や体裁上の不備を正す。
文字や数字ばかりでなく、デザインや発色の確認も行い、特に発色の確認を行う校正を色校正（色校）という。
かつて「校正」の語は古典作品の写本（原文が存在している場合は原文）と別の写本（異本）を照合する「校訂」の意味でも使われた。


In [7]:
!cat ../../data/wiki-ja-train.word | head -5

自然 言語 処理 （ しぜん げんご しょり 、 英語 ： 　 ｎａｔｕｒａｌ 　 ｌａｎｇｕａｇｅ 　 ｐｒｏｃｅｓｓｉｎｇ 、 略称 ： ＮＬＰ ） は 、 人間 が 日常 的 に 使 っ て い る 自然 言語 を コンピュータ に 処理 さ せ る 一連 の 技術 で あ り 、 人工 知能 と 言語 学 の 一 分野 で あ る 。
「 計算 言語 学 」 （ ｃｏｍｐｕｔａｔｉｏｎａｌ 　 ｌｉｎｇｕｉｓｔｉｃｓ ） も 同じ 意味 で あ る が 、 前者 は 工学 的 な 視点 から の 言語 処理 を さ す の に 対 し て 、 後者 は 言語 学 的 視点 を 重視 する 手法 を さ す 事 が 多 い 。
データベース 内 の 情報 を 自然 言語 に 変換 し たり 、 自然 言語 の 文章 を より 形式 的 な （ コンピュータ が 理解 し やす い ） 表現 に 変換 する と い っ た 処理 が 含 ま れ る 。
自然 言語 の 理解 を コンピュータ に さ せ る こと は 、 自然 言語 理解 と さ れ て い る 。
自然 言語 理解 と 、 自然 言語 処理 の 差 は 、 意味 を 扱 う か 、 扱 わ な い か と い う 説 も あ っ た が 、 最近 は 数理 的 な 言語 解析 手法 （ 統計 や 確率 など ） が 広め られ た 為 、 パーサ （ 統語 解析 器 ） など が 一段 と 精度 や 速度 が 上が り 、 その 意味 合い は 違 っ て き て い る 。
cat: write error: Broken pipe


In [8]:
!cat ../../data/wiki-ja-test.word | head -5

校正 （ こうせい 、 英 ： ｐｒｏｏｆ ｒｅａｄｉｎｇ 、 中 ： 校 対 ） は 、 印刷 物 等 の 字句 や 内容 、 体裁 、 色彩 の 誤り や 不 具合 を 、 あらかじめ 修正 する こと 。
校合 （ きょうごう ） と も い う 。
出版 に あた っ て は 、 印刷 に 先立 っ て 仮 刷り を 行 い 、 それ と 原稿 の 内容 を 突き合わせ 、 誤植 や 体裁 上 の 不備 を 正 す 。
文字 や 数字 ばかり で な く 、 デザイン や 発色 の 確認 も 行 い 、 特に 発色 の 確認 を 行 う 校正 を 色 校 正 （ 色校 ） と い う 。
かつて 「 校正 」 の 語 は 古典 作品 の 写本 （ 原文 が 存在 し て い る 場合 は 原文 ） と 別 の 写本 （ 異本 ） を 照合 する 「 校訂 」 の 意味 で も 使 わ れ た 。
